# 1. Import libraries

In [ ]:
import requests 
from bs4 import BeautifulSoup as soup

In [ ]:
# Preventing yourself from getting blocked by TripAdvisor , 
# we will send requests using a random range of time
from random import randint 
from time import sleep

In [ ]:
# help you mix bs4 and xpath 
from lxml import html

In [ ]:
import numpy
import pandas as pd 

# 2. prepare bs4  

In [ ]:
url = 'https://www.tripadvisor.com/Hotels-g293730-Morocco-Hotels.html'
HEADERS = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

# 3. request the home of trip advisor 

In [ ]:
html_doc = requests.get(url,headers=HEADERS)
print(html_doc.status_code)

# 4. parse the html content

In [ ]:
bsobj = soup(html_doc.content,'lxml')

# 5. get hotels names , links , ratings , reviews 

In [ ]:
hotel = {'names':[],'ratings':[],'links':[],'NbrReviews':[]}

In [ ]:
for name in bsobj.findAll('div',{'class':'listing-title'}):
    link = 'https://tripadvisor.in'+name.a['href']
    # Search where is the 1st location of Reviews in links a.find('Reviews')
    posReviews = link.find('Reviews')
    link = link[:posReviews+7] + '-or{}' + link[posReviews+7:]
    
    hotel['names'].append(name.text.strip())
    hotel['links'].append(link)

In [ ]:
for rating in bsobj.findAll('a',{'class':'ui_bubble_rating'}):
    hotel['ratings'].append(rating['alt'].strip())

In [ ]:
for Nbrreview in bsobj.findAll('a',{'class':'review_count'}):
    hotel['NbrReviews'].append(Nbrreview.text.strip())

In [ ]:
# for p in bsobj.findAll('div',{'class':'price-wrap  '}):
#     hotel['price'].append(p.text)

# 6. get the reviews and user info of each hotel

In [ ]:
hotelReviews = {'hotelName':[],'hotelRating':[],'hotelLink':[],'nbrReviews':[],'hotelReview':[],'reviewTitle':[] , 'dateReview':[], 'userName':[] ,'userAddress':[],'contribution':[],'noteUseful':[]}

In [ ]:
# Each page contains 5 reviews => link change dynamically,
# according to the nbr of reviews viewed .

# we will grap only 5 page of reviews for each hotel
pageNbrReviews = [nbr for nbr in range(5,26,5)]

In [ ]:
userName = None
address = None
comment = None
dateComment = None
titleComment = None
dayOfStay = None
contribution = None
noteUseful = None


In [ ]:
print('********')
NbrHotels = len(hotel['names'])
for i in range(0,NbrHotels):
    link = hotel['links'][i]
    for page in pageNbrReviews :
        url = link.format(page)
        html_doc = requests.get(url,headers=HEADERS)
        print(html_doc.status_code)
        sleep(randint(1,8))
        
        #parse using xpath 
        doc = html.fromstring(html_doc.content)
        
        # select all div elements with data-test-target='HR_CC_CARD' = Reviews box
        box_review = doc.xpath("//div[@data-test-target='HR_CC_CARD']")
        
        for div in box_review:
            
            comment = "".join(div.xpath(".//div[2]/div[3]/div[1]/div/span/span/text()"))
            userName = "".join(div.xpath('.//div[1]/div[1]/div[2]/span/a/text()'))
            dateComment= "".join(div.xpath(".//div[1]/div[1]/div[2]/span/text()"))
            titleComment = "".join(div.xpath(".//div[@data-test-target='review-title']/a/span/span/text()"))
            userInfo = div.xpath(".//div[1]/div[1]/div[3]/span")
            dayOfStay = "".join(div.xpath(".//div[2]/div[3]/div[2]/span/text()"))
            
            for info in userInfo:
                info = info.xpath(".//span/text()")
                if(len(info)==1): address = info[0]
                else :
                    info = "".join(info)
                    if 'contribution' in info : contribution = info
                    elif 'helpful' in info: noteUseful = info
            
            hotelReviews['hotelName'].append(hotel['names'][i])
            hotelReviews['hotelLink'].append(hotel['links'][i])
            hotelReviews['hotelRating'].append(hotel['ratings'][i])
            hotelReviews['nbrReviews'].append(hotel['NbrReviews'][i])
            hotelReviews['reviewTitle'].append(titleComment)
            hotelReviews['hotelReview'].append(comment)
            hotelReviews['dateReview'].append(dateComment)
            hotelReviews['contribution'].append(contribution)
            hotelReviews['noteUseful'].append(noteUseful)
            hotelReviews['userName'].append(userName)
            hotelReviews['userAddress'].append(address)
            
            print(hotelReviews)
            
            

# 7. convert the dict to DataFrame then save it  

In [ ]:
df = pd.DataFrame.from_dict(hotelReviews)
df

In [ ]:
df.to_csv('tripAdvisorData.csv')